<a href="https://colab.research.google.com/github/cagBRT/timeSeries/blob/main/11i_Encoder_Decoder_LSTMs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Encoder-Decoder LSTMs**

A model specifically developed for forecasting variable length output sequences is called the Encoder-Decoder LSTM. <br>
The model was designed for prediction problems where there are both input and output sequences, so-called sequence-to-sequence, or seq2seq problems, such as translating text from one language to another. <br>
This model can be used for multi-step time series forecasting. As its name suggests, the model is comprised of two sub-models: the encoder and the decoder.<br><br>
*Jason Bownlee, Machine Learning Mastery*

In [ ]:
from numpy import array
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from keras.utils.vis_utils import plot_model
from tensorflow.keras.layers import LeakyReLU

In [ ]:
 #split a univariate sequence into samples
def split_sequence(sequence, n_steps_in, n_steps_out):
  X, y = list(), list()
  for i in range(len(sequence)):
    # find the end of this pattern
    end_ix = i + n_steps_in
    out_end_ix = end_ix + n_steps_out
    # check if we are beyond the sequence
    if out_end_ix > len(sequence):
      break
    # gather input and output parts of the pattern
    seq_x, seq_y = sequence[i:end_ix], sequence[end_ix:out_end_ix]
    X.append(seq_x)
    y.append(seq_y)
  return array(X), array(y)

In [ ]:
raw_seq = [10, 22, 35, 43, 55, 64, 75, 85, 95]

In [ ]:
# choose a number of time steps
n_steps_in, n_steps_out = 3, 2
# split into samples
X, y = split_sequence(raw_seq, n_steps_in, n_steps_out)
# reshape from [samples, timesteps] into [samples, timesteps, features]
n_features = 1
X = X.reshape((X.shape[0], X.shape[1], n_features))
y = y.reshape((y.shape[0], y.shape[1], n_features))

**Create the model**

**The encoder:**<br>
reads and interperts the input sequence

In [ ]:
model = Sequential()
model.add(LSTM(100, activation='relu', input_shape=(n_steps_in, n_features))) 

**The decoder:**<br>
The output of the encoder is repeated, once for each time step

In [ ]:
model.add(RepeatVector(n_steps_out))

**Define the decoder**

In [ ]:
model.add(LSTM(100, activation='relu', return_sequences=True)) 

Use the same output layers tomake each one-step prediction in the output sequence

In [ ]:
model.add(TimeDistributed(Dense(1)))

**Compile and train the model**

In [ ]:
model.compile(optimizer='adam', loss='mse')
# fit model
model.fit(X, y, epochs=50, verbose=0)

In [ ]:
plot_model(model, show_shapes=True, show_layer_names=True)

**Make a prediction**:<br>
Expect:
106, 120

In [ ]:
x_input = array([75, 85, 95])
x_input = x_input.reshape((1, n_steps_in, n_features))
yhat = model.predict(x_input, verbose=0)
print(yhat)

**Assignment**: 
1. Try different activation functions.(LeakyReLU) Does it change the prediction accuracy?
2. Change the number of epochs, and the input sequence, what is the change in the accuracy?